In [23]:
import pandas as pd

In [24]:
base_file_path = 'data/'

df = pd.read_csv("data/polished3_with_moy_gdp.csv")
df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal', 'Won Medal', 'BMI',
       'GDP'],
      dtype='object')

In [25]:
used_columns = ['NOC', 'Sex', 'Age', 'Height', 'Weight', 'Sport', 'Event','GDP']
old_df = df
df = df[used_columns]

sport_df = df.drop(columns=['Event'])
event_df = df.drop(columns=['Sport'])

In [26]:
# for numerical columns, get a average value for each sport/event having a separated row for Men and Woman
# for categorical columns, get the most common value for each sport/event (in this case is only the NOC country)

def process_sports_data(sports_df, eventOrSport):
    # Identify numerical columns
    numerical_columns = sports_df.select_dtypes(include=['number']).columns

    # Group by Sport and Gender
    def aggregate_func(group):
        averages = group[numerical_columns].mean()
        most_common_noc = group['NOC'].mode()[0] if not group['NOC'].mode().empty else None
        return pd.Series({**averages.to_dict(), 'NOC': most_common_noc})

    result = (
        sports_df
        .groupby([eventOrSport, 'Sex'])
        .apply(aggregate_func)
        .reset_index()
    )

    return result

In [27]:
sport_df = process_sports_data(sport_df, 'Sport')
event_df = process_sports_data(event_df, 'Event')

display(sport_df.head())
display(event_df.head())

/tmp/ipykernel_16864/646806740.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(aggregate_func)
/tmp/ipykernel_16864/646806740.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(aggregate_func)


,Sport,Sex,Age,Height,Weight,GDP,NOC
0,Archery,F,25.694224,167.181200,62.013590,16350.539640,CHN
1,Archery,M,26.795591,178.609218,77.092184,18121.643989,USA
2,Athletics,F,25.271651,169.354536,60.187641,13376.624082,USA
3,Athletics,M,25.635284,180.259659,74.315593,11096.361720,USA
4,Badminton,F,25.200000,168.456061,61.586364,22779.610396,CHN


,Event,Sex,Age,Height,Weight,GDP,NOC
0,Archery Men's Individual,M,27.123100,178.323708,76.656535,16716.524287,USA
1,Archery Men's Team,M,26.161765,179.161765,77.935294,20840.963883,USA
2,Archery Women's Individual,F,26.094340,167.056604,61.758148,15376.566456,CHN
3,Archery Women's Team,F,24.916667,167.423333,62.510000,18243.294195,CHN
4,"Athletics Men's 1,500 metres",M,24.436950,178.281525,65.104106,10565.036228,USA


In [28]:
sport_df.to_csv(base_file_path + 'yourSports.csv', index=False)
event_df.to_csv(base_file_path + 'yourEvents.csv', index=False)

In [29]:
sport_df.to_dict('records')

[{'Sport': 'Archery',
  'Sex': 'F',
  'Age': 25.69422423556059,
  'Height': 167.18120045300114,
  'Weight': 62.013590033975085,
  'GDP': 16350.539640342127,
  'NOC': 'CHN'},
 {'Sport': 'Archery',
  'Sex': 'M',
  'Age': 26.795591182364728,
  'Height': 178.60921843687376,
  'Weight': 77.09218436873748,
  'GDP': 18121.643988908592,
  'NOC': 'USA'},
 {'Sport': 'Athletics',
  'Sex': 'F',
  'Age': 25.27165084646247,
  'Height': 169.35453566885215,
  'Weight': 60.18764066934142,
  'GDP': 13376.624082407812,
  'NOC': 'USA'},
 {'Sport': 'Athletics',
  'Sex': 'M',
  'Age': 25.635284190223615,
  'Height': 180.25965885118438,
  'Weight': 74.31559279127238,
  'GDP': 11096.361719629402,
  'NOC': 'USA'},
 {'Sport': 'Badminton',
  'Sex': 'F',
  'Age': 25.2,
  'Height': 168.45606060606062,
  'Weight': 61.586363636363636,
  'GDP': 22779.610395611136,
  'NOC': 'CHN'},
 {'Sport': 'Badminton',
  'Sex': 'M',
  'Age': 26.364672364672366,
  'Height': 179.63960113960113,
  'Weight': 74.36253561253561,
  'GDP':

In [30]:
def gdp_noc_df(data):
    
    latest_year_data = data[data['Year'] == 2016]

    # Select unique NOC and GDP pairs
    noc_gdp_data = (
        latest_year_data[['NOC', 'GDP']]
        .drop_duplicates()
        .reset_index(drop=True)
    )

    return noc_gdp_data

In [31]:
gdp_and_noc = gdp_noc_df(old_df)
gdp_and_noc

,NOC,GDP
0,ROU,9404.381259
1,ESP,26537.159489
2,ITA,30960.731509
3,AZE,3880.685228
4,FRA,37062.533572
...,...,...
191,SMR,43398.428526
192,TLS,1349.546777
193,SUR,5705.399487
194,TUV,3836.073003
